In [1]:
from pdfparser import Parser
from typing import List, Dict

In [2]:
parser=Parser('test.PDF')
print(parser.header_is_find)

False


In [12]:
def consolidate_tables(self: Parser, pagination: int, table_num: int, own_header:bool):# own_header 表示该页是否有页眉
    def table_is_consecutive(header_msg:list[Dict[str, str]],page_words:list[Dict[str, str]],table_top:str):# 此函数用来判断下一页的第一张表格是否与上一页最后一张表格是同一张表格
        try:
            for word_num,header_word in enumerate(page_words):
                if word_num <len(header_msg):
                    if not header_msg[word_num]['text'] == header_word['text']:
                        return False
                else:
                    if table_top <= header_word['top']:
                        return True
                    else:
                        return False
                    
        except Exception as e:
            print(e)
            return False
    table_statue:Dict[str,list]={
        'pagination':[pagination],
        'list':[table_num]
    }
    header_msg = self.get_header()
    while True:
        if not pagination+1 < self.pages_num:
            break
        page = self.pdf.pages[pagination+1]
        tables = page.find_tables()
        table_top=tables[0].bbox[1]
        page_words=page.extract_words()
        if table_is_consecutive(header_msg=header_msg,page_words=page_words,table_top=table_top):
            table_statue['pagination'].append(pagination+1)
            table_statue['list'].append(1)
        if not len(tables) == 1:
            break
        pagination += 1
    return table_statue


print(consolidate_tables(self=parser, pagination=35, table_num=-1,own_header=True))


{'pagination': [35, 36], 'list': [-1, 1]}
